In [1]:
%pip install pandas numpy matplotlib

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class MeanReversionStrategy:
    def __init__(self, data):
        self.data = data

    def calculate_bollinger_bands(self, window, num_std_dev):
        rolling_mean = self.data['Close'].rolling(window=window).mean()
        rolling_std = self.data['Close'].rolling(window=window).std()
        self.data['UpperBand'] = rolling_mean + (rolling_std * num_std_dev)
        self.data['LowerBand'] = rolling_mean - (rolling_std * num_std_dev)

    def calculate_rsi(self, window):
        delta = self.data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        self.data['RSI'] = 100 - (100 / (1 + rs))

    def generate_bollinger_signals(self):
        self.data['Signal'] = 0.0
        self.data['Signal'] = np.where(self.data['Close'] < self.data['LowerBand'], 1.0, 0.0)
        self.data['Signal'] = np.where(self.data['Close'] > self.data['UpperBand'], -1.0, self.data['Signal'])
        self.data['Position'] = self.data['Signal'].diff()

    def generate_rsi_signals(self, lower_threshold, upper_threshold):
        self.data['Signal'] = 0.0
        self.data['Signal'] = np.where(self.data['RSI'] < lower_threshold, 1.0, 0.0)
        self.data['Signal'] = np.where(self.data['RSI'] > upper_threshold, -1.0, self.data['Signal'])
        self.data['Position'] = self.data['Signal'].diff()

    def plot_bollinger_bands(self):
        plt.figure(figsize=(14, 7))
        plt.plot(self.data['Close'], label='Close Price')
        plt.plot(self.data['UpperBand'], label='Upper Bollinger Band', alpha=0.7)
        plt.plot(self.data['LowerBand'], label='Lower Bollinger Band', alpha=0.7)
        plt.scatter(self.data.index, self.data['Position'] == 1, label='Buy Signal', marker='^', alpha=1, color='green')
        plt.scatter(self.data.index, self.data['Position'] == -1, label='Sell Signal', marker='v', alpha=1, color='red')
        plt.title('Bollinger Bands Strategy Signals')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

    def plot_rsi(self):
        plt.figure(figsize=(14, 7))
        plt.plot(self.data['Close'], label='Close Price')
        plt.plot(self.data['RSI'], label='RSI', alpha=0.7)
        plt.axhline(y=30, color='r', linestyle='--', alpha=0.5)
        plt.axhline(y=70, color='r', linestyle='--', alpha=0.5)
        plt.scatter(self.data.index, self.data['Position'] == 1, label='Buy Signal', marker='^', alpha=1, color='green')
        plt.scatter(self.data.index, self.data['Position'] == -1, label='Sell Signal', marker='v', alpha=1, color='red')
        plt.title('RSI Strategy Signals')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

if __name__ == '__main__':
    # Sample data (replace with your own data)
    data = pd.DataFrame({
        'Close': np.random.rand(100) * 100
    }, index=pd.date_range(start='2022-01-01', periods=100))

    strategy = MeanReversionStrategy(data)

    # Bollinger Bands Strategy
    strategy.calculate_bollinger_bands(window=20, num_std_dev=2)
    strategy.generate_bollinger_signals()
    strategy.plot_bollinger_bands()

    # RSI Strategy
    strategy.calculate_rsi(window=14)
    strategy.generate_rsi_signals(lower_threshold=30, upper_threshold=70)
    strategy.plot_rsi()